In [3]:
module AcitveRecord
  module Validations(base)
    base.extend ClassMethods
  end
  
  module ClassMethods
  end
end

SyntaxError: unexpected keyword_end, expecting end-of-input


`Validations`的实例方法会变成`Base`的实例方法。

`ActiveRecord::Base`包含`Validations`模块时，使用`ActiveRecord::Validations::ClassMethods`模块对`Base`进行了扩展。于是`ClassMethods`中的方法成了`Base`的类方法。

其实只要加一行就能解决问题

extend Validations::ClassMethods

In [5]:
module SecondLevelModule
  def self.included(base)
    base.extend ClassMethods
  end
  
  def second_level_instance_method; 'ok'; end
  
  module ClassMethods
    def second_level_class_method; 'ok'; end
  end
end

module FirstLevelModule
  def self.included(base)
    base.extend ClassMethods
  end
  
  def first_level_instance_method; 'ok'; end
  
  module ClassMethods
    def first_level_class_method; 'ok'; end
  end
  include SecondLevelModule
end

class BaseClass
  include FirstLevelModule
end


BaseClass

In [6]:
BaseClass.new.first_level_instance_method

"ok"

In [8]:
BaseClass.new.second_level_instance_method

"ok"

In [10]:
BaseClass.first_level_class_method

"ok"

In [12]:
BaseClass.second_level_class_method


NoMethodError: undefined method `second_level_class_method' for BaseClass:Class

原来，调用`SecondLevelModule.included`方法时，`base`参数不是`BaseClass`，而是`FirstLevelModule`。结果`SecondLevelModule::ClassMethods`中模块中定义的方法就成了`FirstLevelModule`模块的类方法。

In [14]:
require 'active_support'

module MyConcern
  extend ActiveSupport::Concern
  
  def an_instance_method; "an instance method"; end
  
  module ClassMethods
    def a_class_method; "a class method" end
  end
end

class MyClass
  include MyConcern
end

p MyClass.new.an_instance_method
p MyClass.a_class_method

"an instance method"
"a class method"


"a class method"

`ActiveSupprt::Concern`模块封装了包含并扩展技巧，并且解决了链式包含的问题。一个模块可以通过扩展`Concern`模块并定义自己的`ClassMethods`模块来实现并包含并扩展的功能。


`module#append_feature`方法是Ruby的Ruby的一个内核方法。与`Module#included`方法类似，该方法也是在包含一个模块时被调用。两者之间的区别在于`:included`方法是钩子方法，其默认实现是空的，只有覆写后才有内容。

相反，`append_features`方法包含实际动作，用于检查被包含模块是否已经在包含类的祖先链上，如果不在，则将该模块加入其祖先链。


In [17]:
module M
  def self.append_features(base)
  end
end
class C
  include M
end

C.ancestors

[C, Object, PP::ObjectMixin, Kernel, BasicObject]

上面的代码通过覆写`append_feature`方法，阻止了一个模块被包含的动作。

`append_feature`方法是`Concern`类的一个实例方法，因此在扩展`Concern`的模块中，它会称为一个类方法。

Returns true if mod is a subclass of other. Returns nil if there’s no relationship between the two. (Think of the relationship in terms of the class definition: “class A<B” implies “A<B”).

In [20]:
module ActiveSupport
  module Concern
    def append_features(base)
      if base.instance_variable_defined?(:@_dependencies)
        base.instance_variable_get(:@_dependencies) << self
        return false
      else
        return false if base < self
        @_dependencies.each {|dep| base.send(:include, dep)}
        super
        base.extend_const_get(:ClassMethods)
         if const_defined?(:ClassMethos)

SyntaxError: (pry):137: syntax error, unexpected end-of-input

在一个`concern`中不会包含另外一个`concern`。如果一个`concern`试图包含另一个`concern`，它只是把它们链接到一个依赖图中。如果一个`concern`被一个并非`concern`的模块包含，所有这些以来会一股脑的进入那个包含`concern`的模块中。

在这个作用域中，`self`指向该`concern`，`base`变量则是包含该`concern`的模块。（即可能是一个`concern`，也可能不是
）

在进入`append_feature`方法后，首先检查包含类本身是否是一个`concern`。如果包含类也有`@_dependencies`类变量，那么它也是一个`concern`，在这种情况下，它不会进入包含类的祖先链中，只会被添加到依赖列表里。该代码同时返回`false`，用来指明模块并没有真正被包含。例如，当`ActiveModel::Validations`模块被`ActiveRecord::Validations`包含时，就会发生这种情况。

如果包含类不是一个`concern`，例如，当`ActiveRecord::Validations`被`ActiveRecord::Base`所包含时，会发生什么呢？在这种情况下，首先要检查`concern`是否已经出现在类的祖先链中，这种情况在链式包含时有可能发生（这就是代码`base < self`所包含的意思）。如果没有出现在祖先链中，那么会进入整个代码最关键的时刻；`concern`中的依赖会被递归包含类中。这种最小化的依赖解决方式可以解决“链式包含的问题”。